In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F



# test random code
x = torch.empty(3,3)
z = torch.randn(5,5)
y = torch.tensor([[3,3],[4,9]])
print(x)
print(y)
print(z)

tensor([[3.7971e+35, 5.4791e-43, 2.3694e-38],
        [2.3694e-38, 2.3694e-38, 2.3694e-38],
        [1.4013e-45, 5.2452e-06, 0.0000e+00]])
tensor([[3, 3],
        [4, 9]])
tensor([[ 0.1672, -0.6091, -0.8159,  0.9196, -0.3130],
        [ 0.0839, -0.6435,  1.5020, -0.8243,  0.7386],
        [-1.7716,  0.5840,  1.3356,  0.2318,  0.8519],
        [ 0.9672, -0.3782, -0.6670, -2.4481, -0.0496],
        [ 0.5110, -0.3785, -0.7690,  0.5420, -0.0176]])


In [47]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()

print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [28]:
# The learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [33]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0432,  0.0469,  0.0131,  0.0828, -0.0261, -0.1214, -0.0997, -0.0568,
         -0.0157, -0.1355]], grad_fn=<AddmmBackward>)


In [34]:
net.zero_grad()
out.backward(torch.randn(1, 10))


In [35]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(2.3834, grad_fn=<MseLossBackward>)


In [36]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU